`GB` BigData / [Олег Гладкий](https://gb.ru/users/3837199) // домашнее задание

`262698` __Методы сбора и обработки данных из сети Интернет__:  `02`. Парсинг даных: HTML, DOM, __XPath__

# XPath: Парсер новостного сайта

Написать приложение или функцию, которые собирают основные новости с сайта на выбор:
 * lenta.ru, 
 * yandex-новости. 
 
#### Получаемы данные
Для парсинга использовать `XPath`. Структура данных в виде словаря должна содержать:
- *название источника;
- наименование новости;
- ссылку на новость;
- дата публикации.

Минимум один сайт, максимум — все два


#### Проверка расположения интерпретатора Питона

In [66]:
!where python

C:\Programs\Anaconda3\python.exe
C:\Program Files\Python38\python.exe


#### Включаемы модули

In [3]:
import pandas as pd
from pprint import pprint 
import datetime

import requests
from lxml import html

## 1. Лента.РУ
`XPath` — Парсинг при его помощи!

### Забираем все данные с сайта `Lenta.ru`

 Обращаемся к сайту как Хром методом `get` модуля `requests`. Кроме того:
 * необходимо запомнить дату текущего запроса
 * время этого текущего запроса
 * для продолжения работы необходимо уточнить код ответа сервера `status_code`
 * а так же необходимо уточнить, вернул ли нам сервер данные в формате `html`, информцию об этом сервер возвращает в заголовках `headers` в параметре `'Content-Type'`
 

In [26]:
%%time
url = 'https://lenta.ru/'

headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/105.0.0.0 Safari/537.36',
}

response = requests.get(url=url, headers=headers)

news_date = str(datetime.date.today())
news_time = str(datetime.datetime.now().strftime("%H:%M"))

response.status_code, response.headers['Content-Type'], response.url, 

Wall time: 95.7 ms


(200, 'text/html; charset=utf-8', 'https://lenta.ru/')

#### Перобразуем данные к формату `DOM`

Преобразование тела документа как строки в ответе `response.text` в _дерево_ `DOM` при помощи метода `fromstring` модуля `html` 

In [27]:
dom = html.fromstring(response.text) 

### Парсим
Выберем из дерева `DOM` только блоки с новостями. Каждый такой блок новостей — это объект. Все объекты объеденены в список `list`. При этом самым верхним уровнем в структуре для такого объекта будет не начало сайта или дерева `DOM`, а начало этого информационного блока — тэг `a`.

#### Функция
Фукнция для выборки парснутых данных в новостные элементы 

In [28]:
def eject(elements):
    ''' забираем первый элемент из списка'''
    if isinstance(elements, list):
        if len(elements) == 1 or len(elements) > 1:
            element = str(elements[0]).replace('\xa0', ' ')
        else:
            element = None
    return element

#### Блоки МИНИ 
Класс `card-mini__title` с минимальными новостями из нашего `DOM` в виде списка объектов:
* Название новости
* Ссылку
* Время на текущий день. 
    - Новостной блок не содержит дату, а только время. Но мы располагаем датой запроса этих новостей, которую мы сохранили сразу после запроса в переменной `news_date`. 
    - Если время отсутствует в блоке с новостью, то воспользуемся временем запроса к сайту, сохранённого в переменной `news_time`.

__Блоки МИНИ__: все объекты с минимальными новостями

In [31]:
news_mini = dom.xpath('//a[contains(@class, "card-mini")]')
print(f"Блоки МИНИ: количество: {len(news_mini)}")

Блоки МИНИ: количество: 87


__Блоки МИНИ__: Парсим данные для кадой новости и сохраняем их как строку `DataFrame`.

In [32]:
news_mini_table = pd.DataFrame(columns=['Name', 'Link', 'Date', 'Source', 'N_Type']) # инициализирум выходную таблицу
news_type = "mini"

for i in news_mini:
    name = eject(i.xpath('.//div[contains(@class, "card-mini__text")]/span[contains(@class, "card-mini__title")]/text()'))
    link = url.strip('/') + eject(i.xpath('.//@href'))
    time = eject(i.xpath('.//div[contains(@class, "card-mini__text")]/div[contains(@class, "card-mini__info")]/time[contains(@class, "card-mini__date")]/text()'))
    
    date = news_date + ';' + (time if time else news_time) 

    news_mini_table = news_mini_table.append({'Date': date, 'Name': name, 'Link': link, 'N_Type': news_type}, ignore_index=True)

print(f"НОВОСТИ МИНИ: {len(news_mini_table)} позиция распарсена")

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

# news_mini_table

НОВОСТИ МИНИ: 87 позиция распарсена


#### Блоки БИГ
Новостные блоки БИГ: класс объектов `card-mini__big` с расширенными новостями из нашего `DOM` в виде 2-ого списка `list`.

In [33]:
news_big = dom.xpath('//a[contains(@class, "card-big")]')
print(f"Блоки БИГ, количество: {len(news_big)} объекта получено")

Блоки БИГ, количество: 52 объекта получено


Источник Source, переменная `src`: распарсить значение атрибута `xlink:href` тега `use`не получиться при помощи `xpath`, потому что, это не `htmp`. А это `xml`, который `XPATH` не понимает! 

Для доступа к этому «внешнему» тэгу `use` нужно обратиться к нему при помощи функции `name()` как `/*[name()="use"]` и обратиться к значению атрибута этого xml-тэга при помощи указателя `attribute::*` (это наверное указатель на любой атрибут).

In [34]:
news_big_table = pd.DataFrame(columns=['Name', 'Link', 'Date', 'Source', 'N_Type'], index=[]) # инициализирум выходную таблицу
news_type = "BIG"

for i in news_big:
    title = eject(i.xpath('.//div[contains(@class, "card-big__titles")]/h3[contains(@class, "card-big__title")]/text()'))
    notes = eject(i.xpath('.//div[contains(@class, "card-big__titles")]/span[@class]/text()'))
    time =  eject(i.xpath('.//div[contains(@class, "card-big__info")]/time[contains(@class, "card-big__date")]/text()'))
    link =  eject(i.xpath('.//@href'))
    
    # ??? Как получить доступ к значению атрибута xlink:href в ссылке xlink:href="#ui-label_motor" ???
    
    src =   eject(i.xpath('.//div[contains(@class, "card-big__info")]/*[name()="svg"]/*[name()="use"]/attribute::*'))   

    name  = title + (notes if notes else "")
    date = time
    link = url.strip("/") + link
    
    news_big_table = news_big_table.append({'Name': name, 'Link': link, 'Date': date, 'Source': src, 'N_Type': news_type}, ignore_index=True)

    
print(f"НОВОСТИ БИГ: {len(news_big_table)} позиция распарсена")
# news_big_table

НОВОСТИ БИГ: 52 позиция распарсена


### ВСЕ НОВОСТИ: МИНИ + БИГ

In [35]:
news_table = news_mini_table.append(news_big_table, ignore_index=True)

In [36]:
print(f"ВСЕГО новостей: {len(news_table)}")
news_table

ВСЕГО новостей: 139


,Name,Link,Date,Source,N_Type
0,Британцев разозлила постановка «Ромео и Джулье...,https://lenta.ru/news/2022/11/03/nebinar/,2022-11-03;22:07,NaN,mini
1,Турция прокомментировал процесс вступления Фин...,https://lenta.ru/news/2022/11/03/nato/,2022-11-03;22:06,NaN,mini
2,Сотрудники посольства США посетили баскетболит...,https://lenta.ru/news/2022/11/03/price/,2022-11-03;22:05,NaN,mini
3,Экс-возлюбленная Тимати показала лицо после ум...,https://lenta.ruhttps://moslenta.ru/news/lyudi...,2022-11-03;21:57,NaN,mini
4,Британия запретит своим компаниям перевозку ро...,https://lenta.ru/news/2022/11/03/potolok/,2022-11-03;21:53,NaN,mini
5,Иностранцы сократили финансовую помощь Украине,https://lenta.ru/news/2022/11/03/sokratili/,2022-11-03;21:51,NaN,mini
6,В Латвии возмутились меню на русском языке в р...,https://lenta.ru/news/2022/11/03/menu/,2022-11-03;21:45,NaN,mini
7,Лавров прибыл с визитом в ОАЭ,https://lenta.ru/news/2022/11/03/lavrovoae/,2022-11-03;21:40,NaN,mini
8,В НАТО заявили о выполнении Финляндией и Швеци...,https://lenta.ru/news/2022/11/03/usloviya/,2022-11-03;21:38,NaN,mini
9,Два российских министра остались без света на ...,https://lenta.ru/news/2022/11/03/svetttt/,2022-11-03;21:37,NaN,mini


<!--  -->

__P.S.__

Не удалось распарсить значение атрибута `xlink:href` тега `use` — как только не пытался при помощи `xpath`... Потому что, это не `html`, а `xml`, который `XPATH` не разбирает.

Подсмотрел...

<!--  -->

## 2. Yandex-новости
`XPath`. (Работа после сдачи ДЗ)

Теперь это dzen
Для DZEN требуется авторизация для подбора контента под пользователя. И Дзен, вместо новостей, перенаправляет на страничку авторизации.

Чтобы избежать авторизации и получить универсальный контент, нужно передать параметр `'sso_failed'` со значением — пустая строка (`""`)

In [8]:
%%time
url = 'https://yandex.ru/news/'

headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/105.0.0.0 Safari/537.36',
}

params = {
    'sso_failed': '',
}

response = requests.get(url=url, headers=headers, params=params)
dom = html.fromstring(response.text)

news_date = str(datetime.date.today())
news_time = str(datetime.datetime.now().strftime("%H:%M"))

response.status_code, response.headers['Content-Type'], response.url, 

Wall time: 357 ms


(200,
 'text/html; charset=UTF-8',
 'https://dzen.ru/news/?issue_tld=ru&sso_failed=')

In [12]:
# response.text

#### Все объекты-контейнеры с новостью

In [9]:
news_dzen = dom.xpath('.//div[contains(@class, "mg-card ")]')
print(f"Блоки DZEN: количество: {len(news_dzen)}")

Блоки DZEN: количество: 65


In [10]:
for i in news_dzen:
    name = eject(i.xpath('.//h2/a/text()'))
    #     text = i.xpath('.//div[contains(@class, "mg-card__text-content")]/div[contains(@class,"mg-card__text")]/h2/a/text()')  # [0].replace('\xa0', ' ')
    print(name)
    
    

#     name = i.xpath('.//h2[contains(@class, "mg-card__title")]/a/text()')    
#     name = i.xpath('.//div[contains(@class, "mg-card__text")]/div[contains(@class, "mg-card__annotation")]/text()')    

Главе Чечни Кадырову присвоили звание генерал-полковника
Правозащитник Чиков заявил о первом уголовном деле за уклонение от мобилизации в России
Путин признался, что результаты референдумов на новых территориях его порадовали и удивили
Минобороны: на Андреево-Криворожском направлении войска удерживают занимаемые позиции
Песков опроверг сообщения о новом обращении Путина по смене статуса спецоперации
Подмосковный педагог стал победителем конкурса «Учитель года России – 2022»
Андрей Воробьев поздравил учителей с профессиональным праздником и вручил награды
Полиция задержала подозреваемого в сбыте примерно 1 кг кокаина в Клину
Замминистра обороны РФ напутствовал мобилизованных в Коломне
Подозреваемого в краже кофе и зубной пасты задержали в Коломне
Катапульта отправила в небо спутники НАСА
Актер Стас Ярушин поделился историей создания песни «Я и ты» для телесериала «Универ»
Spiked: консерваторы из Британии бьются в «предсмертной агонии»
Путин подписал указ об отсрочке от частичной мобилиз

## 3. books.toscrape.com
`XPath` (Работа после сдачи ДЗ)

Учебный сайт для парсинга, объект парсинга — книги...

In [6]:
%%time
url = 'http://books.toscrape.com/'

headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/105.0.0.0 Safari/537.36',
}
response = requests.get(url=url, headers=headers)

CPU times: total: 46.9 ms
Wall time: 854 ms


In [7]:
response.status_code, response.headers['Content-Type'], response.url, response.encoding

(200, 'text/html', 'http://books.toscrape.com/', 'ISO-8859-1')

Изменим кодировку на __Юникод__:

In [8]:
response.encoding = 'utf-8'

In [9]:
dom = html.fromstring(response.text)

In [10]:
books = dom.xpath('//ol[@class="row"]/li')
len(books)

20

In [11]:
books_table = pd.DataFrame()
book_info = {}

for book in books:
    title = book.xpath('.//h3/a/@title')[0]
    price = book.xpath('.//p[@class="price_color"]/text()')[0]
#     .decode('utf8')
#     .encode('utf8')
    
    book_info = {
        'title': title,
        'price': price,
    }
    book_info = pd.DataFrame([book_info])
    books_table = pd.concat([books_table, book_info], axis=0, ignore_index=True)


books_table

,title,price
0,A Light in the Attic,£51.77
1,Tipping the Velvet,£53.74
2,Soumission,£50.10
3,Sharp Objects,£47.82
4,Sapiens: A Brief History of Humankind,£54.23
5,The Requiem Red,£22.65
6,The Dirty Little Secrets of Getting Your Dream...,£33.34
7,The Coming Woman: A Novel Based on the Life of...,£17.93
8,The Boys in the Boat: Nine Americans and Their...,£22.60
9,The Black Maria,£52.15
